<a href="https://colab.research.google.com/github/Shilpar31/Port-folio/blob/master/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2610400 sha256=de91e62a27cd0c6526e2f6a0c6d2eca6f3036caf32641cca32b369fb31af0880
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [1]:
!pip install numpy==1.26.4
!pip install scikit-surprise
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

In [1]:
from google.colab import files
uploaded = files.upload()  # choose ratings.csv and movies.csv

Saving movies.csv to movies.csv
Saving ratings.csv to ratings.csv


In [4]:
import pandas as pd
rating=pd.read_csv("ratings.csv")
movie=pd.read_csv("movies.csv")
rating.head(10)
#print(movie.head(10))

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [5]:
# Check for null values
print(rating.isnull().sum())
print(movie.isnull().sum())


userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
movieId    0
title      0
genres     0
dtype: int64


In [3]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import joblib

# Load ratings.csv
ratings = pd.read_csv("ratings.csv")

# Surprise needs a Reader object
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train SVD
model = SVD(n_factors=50, n_epochs=20, verbose=True)
model.fit(trainset)

# Evaluate
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)

# Save model
joblib.dump(model, "svd_model.pkl")
print("✅ Model trained and saved!")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
RMSE: 0.8782
✅ Model trained and saved!


In [11]:
def recommend_movies(user_id, n=5):
    movie_ids = ratings['movieId'].unique()
    movies = pd.read_csv("movies.csv")

    # Predict ratings for all movies not yet rated
    predictions = [ (mid, model.predict(user_id, mid).est) for mid in movie_ids ]

    # Sort by estimated rating
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Top N
    top_n = predictions[:n]

    # Map to movie titles
    recommended = [ movies[movies['movieId']==mid]['title'].values[0] for mid, _ in top_n ]
    return recommended

# Example
print(recommend_movies(1, 5))

['Usual Suspects, The (1995)', 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Fight Club (1999)', 'Shawshank Redemption, The (1994)', 'Rear Window (1954)']


In [5]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
from surprise import SVD

# Load model + dataset
model = joblib.load("svd_model.pkl")
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

st.title("🎬 Movie Recommender (SVD)")

user_id = st.number_input("Enter User ID:", min_value=int(ratings['userId'].min()),
                          max_value=int(ratings['userId'].max()), value=1)

if st.button("Get Recommendations"):
    movie_ids = ratings['movieId'].unique()
    predictions = [ (mid, model.predict(user_id, mid).est) for mid in movie_ids ]
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_n = predictions[:5]
    recommended = [ movies[movies['movieId']==mid]['title'].values[0] for mid, _ in top_n ]

    st.write("Top 5 Recommendations:")
    for i, movie in enumerate(recommended, 1):
        st.write(f"{i}. {movie}")


Writing app.py


In [13]:
#!ngrok config add-authtoken 32TJ7R52bRN48cZnl9rwvGzbDIv_6huEHsPcTE5nFEKVYs93u
!ngrok config add-authtoken 32TJ7R52bRN48cZnl9rwvGzbDIv_6huEHsPcTE5nFEKVYs93u

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
!pkill -f ngrok

In [15]:
!streamlit run app.py &
from pyngrok import ngrok

# Connect to the Streamlit app running on port 8501
public_url = ngrok.connect(8501, "http")
print("🌍 Your app is live here:", public_url)


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
🌍 Your app is live here: NgrokTunnel: "https://9b7ba1cafabb.ngrok-free.app" -> "http://localhost:8501"
